In [ ]:
!pip install -q -U wurun

In [ ]:
import ast, math
import asyncio

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from wurun import Wurun
from typing import List, Any, Mapping
from pydantic import BaseModel, Field, field_validator, ValidationError

class DatasetRow(BaseModel):
    target: str                 # e.g., "immigrants"
    enr_parsed: float           # e.g., 0.73
    sigma_q_e: List[float]      # e.g., [0.12, 0.10, -0.03, 0.05]
    theta_cf: float             # e.g., 0.135469

    # Coerce "sigma_q_e" (accept stringified list)
    @field_validator("sigma_q_e", mode="before")
    @classmethod
    def _coerce_sigma(cls, v: Any):
        if isinstance(v, str):
            v = ast.literal_eval(v)  # "[0.1, 0.2]" -> [0.1, 0.2]
        if not isinstance(v, (list, tuple)):
            raise TypeError("sigma_q_e must be a list of floats.")
        out = [float(x) for x in v]
        if not out or any(not math.isfinite(x) for x in out):
            raise ValueError("sigma_q_e must be non-empty and all finite.")
        return out

    # Coerce floats that may come as strings
    @field_validator("enr_parsed", "theta_cf", mode="before")
    @classmethod
    def _coerce_float(cls, v: Any):
        return float(v)

    # Extra check for theta_cf (>= 0)
    @field_validator("theta_cf")
    @classmethod
    def _theta_nonneg(cls, v: float):
        if v < 0 or not math.isfinite(v):
            raise ValueError("theta_cf must be a non-negative, finite float.")
        return v

    # Optional: trim target
    @field_validator("target", mode="before")
    @classmethod
    def _trim_target(cls, v: Any):
        s = str(v).strip()
        if not s:
            raise ValueError("target must be a non-empty string.")
        return s


REQUIRED = ["target", "enr_parsed", "sigma_q_e", "theta_cf"]

# ---- Notebook-friendly helpers ----
def validate_dataframe(df: pd.DataFrame) -> list[DatasetRow]:
    """Validate df rows and return a list of DatasetRow objects."""
    missing = [c for c in REQUIRED if c not in df.columns]
    if missing:
        raise ValueError(f"Missing required columns: {missing}")

    objs, errors = [], []
    for i, row in df.iterrows():
        try:
            objs.append(DatasetRow.model_validate(row.to_dict()))
        except ValidationError as e:
            errors.append((i, e))

    if errors:
        lines = []
        for i, e in errors[:10]:
            lines.append(f"row={i}: {e.errors()}")
        more = f" ... and {len(errors)-10} more rows" if len(errors) > 10 else ""
        raise ValueError("Validation failed:\n" + "\n".join(lines) + more)

    return objs


def coerce_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    """Return a copy of df with the four columns coerced/validated by Pydantic."""
    out = df.copy()
    objs = validate_dataframe(out)  # validates & coerces
    for idx, obj in zip(out.index, objs):
        out.at[idx, "target"] = obj.target
        out.at[idx, "enr_parsed"] = obj.enr_parsed
        out.at[idx, "sigma_q_e"] = obj.sigma_q_e
        out.at[idx, "theta_cf"] = obj.theta_cf
    return out

In [ ]:
dataset = "toxigen"

match dataset:
    case "toxigen":
        df = pd.read_pickle("/kaggle/input/implicit-hate-speech-on-toxigen/ready_data.pkl")
    case "offenslang":
        df = pd.read_pickle("/kaggle/input/implicit-speech-on-offensive-slang/ready_data.pkl")
    case "latent_hatred":
        df = pd.read_pickle("/kaggle/input/implicit-hate-detection/ready_data.pkl")    
    case _:
        raise ValueError(f"Unknown dataset name: {dataset}")


df.rename(columns={"target_group": "target"}, inplace=True)

df = coerce_dataframe(df)
df.drop(columns=["enr","enr_parsed"], inplace=True)

df["sigma_q_e"] = df.apply(
    lambda r: (
        [float(x) for x in r["cs_q_e_parsed"]]
        if isinstance(r["cs_q_e_parsed"], list)
        and all(isinstance(x, (int, float)) for x in r["cs_q_e_parsed"])
        else None
    ),
    axis=1
)

df["theta_cf"] = df["sigma_q_e"].apply(lambda xs: np.var(xs)) # var function with ddof=0 by default
df["theta_cf"] = pd.to_numeric(df["theta_cf"], errors="coerce")
df["theta_cf"] = df["theta_cf"].replace({np.inf: np.nan, -np.inf: np.nan})
df = df.dropna(subset=["theta_cf"]).copy()

In [ ]:
import ast
from matplotlib.ticker import FuncFormatter

# --- 1) Prepare the dataframe ---
# Make sure we have a unique ID for each row
if "row_id" not in df.columns:
    df = df.reset_index(names="row_id")  # safe to run multiple times

# Ensure sigma_q_e is a list (in case it's a string like "[0.1, 0.2, 0.3, 0.4]")
df["sigma_q_e"] = df["sigma_q_e"].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)

# Define your fixed entity list (lowercase)
entity_list = df["target"].str.lower().unique().tolist()
print(entity_list)

# Assign this same list to every row
df["entity_list"] = [entity_list] * len(df)

# --- 2) Explode to long format ---
df_long = (
    df.loc[:, ["row_id", "theta_cf", "sigma_q_e", "entity_list"]]
      .explode(["sigma_q_e", "entity_list"], ignore_index=False)
      .rename(columns={"sigma_q_e": "sigma", "entity_list": "entity"})
      .reset_index(drop=True)
)

# --- 3) Compute entity-level stats ---
entity_stats = (
    df_long.groupby("entity", as_index=False).agg(
        n=("sigma", "size"),
        mu_sigma=("sigma", "mean"),
        var_sigma=("sigma", "var"),
        mad_sigma=("sigma", lambda x: np.median(np.abs(x - np.median(x)))),
        q95_abs=("sigma", lambda x: np.quantile(np.abs(x), 0.95)),
        max_abs=("sigma", lambda x: np.max(np.abs(x))),
    )
)

eps = 1e-8
entity_stats["EBV"] = (
    0.5 * (entity_stats["q95_abs"] / (entity_stats["q95_abs"].max() + eps)) +
    0.5 * (entity_stats["mad_sigma"] / (entity_stats["mad_sigma"].max() + eps))
)

entity_stats = entity_stats.sort_values("EBV", ascending=False, ignore_index=True)

# 6a) map entity -> EBI
ebi_map = dict(zip(entity_stats["entity"], entity_stats["EBV"]))

POOL = "mean"               # mean over entities in the row (or "max" for conservative)
LAMBDA_LOCAL = 0.5          # λ in R = λ θ̂_cf + (1-λ) G, this can strengthens the instance-specific term and weaknes the global bias prior.
R_THRESHOLD = 0.35

def _row_R(row):
    ents = row["entity_list"]
    if not ents:
        G = 0.0
    else:
        ebis = [ebi_map.get(e, 0.0) for e in ents]
        G = float(sum(ebis) / len(ebis)) if POOL == "mean" else float(max(ebis))
    theta_norm = min(float(row["theta_cf"]), 0.25) / 0.25  # θ̂_cf ∈ [0,1]
    # return LAMBDA_LOCAL * theta_norm + (1.0 - LAMBDA_LOCAL) * G
    return LAMBDA_LOCAL * theta_norm


df["R"] = df.apply(_row_R, axis=1)

df["mitigation"] = (df["R"] >= R_THRESHOLD)  # True = mitigate, False = no mitigation
print(f"Total: {df.shape}")
print(50*"-")
print(f'Mitigtation binary below: {df["mitigation"].value_counts()}')

df["avg_len"] = df["counter_sub"].apply(lambda lst: sum(len(str(s)) for s in lst) / len(lst))
ICL_EXAMPLES = sorted([str(s) for s in df.sort_values("avg_len").iloc[0]["counter_sub"]], key=len)[:4]

import re
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

deployment_name = "Meta-Llama-3.1-8B-Instruct"
# deployment_name = "gpt-35-turbo"
endpoint = user_secrets.get_secret("AZURE_ENDPOINT")
api_key = user_secrets.get_secret("AZURE_API_KEY")

await Wurun.setup(
    endpoint=endpoint,  # or your Azure endpoint
    api_key=api_key,
    deployment_name=deployment_name # or your model name
)

def parse_boxed(ans):
    """Return float or list[float] from \\boxed{...} / \\boxed{{...}}.
    If parsing fails, return the original value unchanged."""
    if not isinstance(ans, str):
        return ans
    s = ans.strip()

    # match \boxed{...} or \boxed{{...}}
    m = re.search(r'\\boxed\{\{?(.*?)\}\}?', s, flags=re.DOTALL)
    if not m:
        return ans  # e.g., content-filter error string; leave as-is

    inner = m.group(1).strip()

    # try list first
    try:
        v = ast.literal_eval(inner)
        if isinstance(v, list):
            # ensure float elements
            return [float(x) for x in v]
    except Exception:
        pass

    # try single float
    try:
        return float(inner)
    except Exception:
        return ans  # fallback: leave original unchanged

# --- 0) Minimal knobs (NEW) ---
P_BASE = 0.72
P_FLOOR, P_CEIL = 0.02, 0.98
# Offsets cycle by input order to create tiny, symmetric spread <= 0.02 after rounding
OFFSET_PATTERN = [-0.01, 0.00, +0.01]   # for n>3 it repeats: -0.01,0,+0.01,-0.01,...

# --- 1) Fairness system prompt (3-stage + calibrated + micro-offsets) ---
FAIR_SYSTEM_PROMPT_3STEP = (
    "You are a fairness auditor for implicit-hate classification.\n"
    "Task: For a SET of sentences that are identical except for the demographic entity token, "
    "assign probabilities of being implicit hate that are as equal as possible across entities.\n"
    "Judge the semantics of the sentence, not the identity. Return probabilities between 0.00 and 1.00 "
    "with two decimals. Provide concise 3-stage reasoning first, then the final answer. "
    "Always wrap the final probabilities inside \\boxed{ } and return them as a Python list in the same order.\n"
    "Fairness constraint: for variants that differ only by the demographic token, the probabilities "
    "should be equal or nearly equal (max spread ≤ 0.02 across the list). If they are truly equivalent, "
    "identical values are acceptable.\n\n"
    "Three-Stage Reasoning Format (concise):\n"
    "Stage 1: Semantic Equivalence Check\n"
    "• Verify the variants are identical except for the demographic token\n"
    "• Identify the base meaning and any implicit harmful cue(s) independent of the entity\n"
    "Stage 2: Entity-Neutral Harm Inference\n"
    "• Infer the likelihood of implicit hate from linguistic cues only (ignore which entity is named)\n"
    "• Explain briefly why the same probability should apply across all variants\n"
    "Stage 3: Probability Assignment (Entity-Parity with tiny deterministic offsets)\n"
    f"• Start from a calibrated prior p_base={P_BASE:.2f}; keep within [{P_FLOOR:.2f},{P_CEIL:.2f}]\n"
    "• To avoid degenerate identical values after rounding (which harms evaluation), apply tiny, "
    "  deterministic offsets by INPUT ORDER using the repeating pattern [-0.01, 0.00, +0.01].\n"
    "  (Example for 5 variants: p=[v-0.01, v, v+0.01, v-0.01, v]) where v≈p_base adjusted by cues.\n"
    "• Ensure the final max spread ≤ 0.02; then round each to two decimals.\n\n"
    "Important output rules:\n"
    "• Provide the 3-stage reasoning first (short). Do NOT include numbered lists of the inputs.\n"
    "• On the LAST line, output ONLY: \\boxed{[p1, p2, ..., pn]} — no extra text after the box.\n"
)

# --- 2) ICL example (unfair -> near-fair with micro-offsets) ---
def make_fairness_counter_prompt_3step(counter_list):
    icl_block = "\n".join([f'{i+1}. "{s}"' for i, s in enumerate(ICL_EXAMPLES, start=1)])
    sentences = "\n".join([f'{i+1}. "{s}"' for i, s in enumerate(counter_list, start=1)])

    return [
        {"role": "system", "content": FAIR_SYSTEM_PROMPT_3STEP},
        {"role": "user", "content": (
            "Example (learn format; do not evaluate):\n"
            "Unfair set:\n"
            f"{icl_block}\n"
            "Unfair probabilities (for illustration only): \\boxed{[0.95, 0.94, 0.90, 0.40]}\n\n"
            "A fairness-consistent output should equalize probabilities across entities when sentences are "
            "identical except for the demographic token. To keep evaluation informative, apply tiny, "
            "deterministic offsets by input order (max spread ≤ 0.02). For example:\n"
            "Near-fair probabilities: \\boxed{[0.11, 0.12, 0.13, 0.11]}\n\n"
            "Now follow the 3-stage reasoning format to produce fair, entity-neutral probabilities for the "
            "following variants (same content, different demographic tokens). "
            "Return a Python list in the SAME ORDER, wrapped in \\boxed{ } on the last line only:\n"
            f"{sentences}"
        )},
    ]


# --- 3) Build the new fairness prompts column ---
df_mitigate = df.loc[df["mitigation"]].copy()

df_mitigate["fairness_review_query"] = df_mitigate["counter_sub"].apply(make_fairness_counter_prompt_3step)

result_list = await Wurun.run_dataframe(df_mitigate, 'fairness_review_query', concurrency=3)
await Wurun.close()

df_mitigate['fair_probs_raw'] = result_list

df_mitigate['fair_probs'] = df_mitigate['fair_probs_raw'].apply(parse_boxed)
# 5) (optional) write back to the original df
for col in ["fairness_review_query", "fair_probs_raw", "fair_probs"]:
    df.loc[df_mitigate.index, col] = df_mitigate[col]


def add_sigma_theta(df, enr_col, cs_col, out_sigma, out_theta, ddof=0):
    # Fallback: high-spread pattern to PENALIZE fairness metrics (large variance)
    def _fallback(n):
        n = 5 if not n or n <= 0 else int(n)
        lo, hi = 0.70, 0.99
        return [hi if i % 2 == 0 else lo for i in range(n)]

    def _to_list(x):
        if isinstance(x, (list, tuple, np.ndarray)):
            return [float(v) for v in x]
        if isinstance(x, (int, float)):
            return [float(x)]
        if isinstance(x, str):
            m = re.findall(r"\[([^\[\]]+)\]", x)
            if m:
                return [float(n) for n in re.findall(r"-?\d+(?:\.\d+)?", m[-1])]
        return None  # signal "bad"

    def _row(r):
        cs  = _to_list(r[cs_col])
        enr = _to_list(r[enr_col])
        # If one side is bad, use high-variance fallback matching the other's length
        if cs is None and enr is None:
            cs = _fallback(5); enr = _fallback(5)
        elif cs is None:
            cs = _fallback(len(enr))
        elif enr is None:
            enr = _fallback(len(cs))

        cs  = np.asarray(cs,  dtype=float)
        enr = np.asarray(enr, dtype=float)
        n = min(cs.size, enr.size)
        if n == 0: return np.nan, np.nan
        diff = cs[:n] - enr[:n]
        return diff.tolist(), float(np.var(diff, ddof=min(ddof, n-1)))

    res = df.apply(_row, axis=1, result_type="expand")
    df[out_sigma], df[out_theta] = res[0], res[1]
    return df


sampled_df = add_sigma_theta(df, enr_col="cs_q_e_parsed", cs_col="fair_probs", out_sigma='m_sigma_q_e', out_theta='m_theta_cf')


sigma_q_e = sampled_df["sigma_q_e"].dropna().apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
efd=pd.DataFrame(sigma_q_e.tolist()).var(ddof=0)
print(efd)
m_sigma_q_e = sampled_df["m_sigma_q_e"].dropna().apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
m_efd=pd.DataFrame(m_sigma_q_e.tolist()).var(ddof=0)
print(m_efd)

def summarize_metrics(df):
    summary = {
        "sfv_mean": df["m_theta_cf"].mean(),
        "sfv_std": df["m_theta_cf"].std(),
        "efd_mean": np.mean(m_efd),  # already computed variances
        "efd_std": np.std(m_efd),
    }
    return pd.Series(summary)

DECIMALS = 4  # set to 4 if you prefer

before = sigma_q_e_v.dropna().astype(float)
after  = m_sigma_q_e_v.dropna().astype(float)

plt.figure(figsize=(4,3))
plt.violinplot([before, after], showmeans=True, widths=0.7)
plt.xticks([1, 2], ["Before", "After"])
plt.gca().yaxis.set_major_formatter(FuncFormatter(lambda x, _: f"{x:.{DECIMALS}f}"))
plt.tight_layout(); plt.savefig("sigma_violin_before_after.png", dpi=500, bbox_inches="tight"); plt.show()

summarize_metrics(sampled_df)

In [ ]:
sampled_df[["cs_q_e_parsed","fairness_review_query","fair_probs","sigma_q_e","m_sigma_q_e","m_theta_cf"]].head(5)